In [655]:
from qutip import *
from numpy import pi, linalg
from scipy.special import jv

In [656]:
si = qeye(2)

sx = sigmax()

sy = sigmay()

sz = sigmaz()

n = 1

m = 1

extmagfield_m = 1

extmagfield_n = [[0, 0, 0],[0, 0, 0],[0.3, 0.3, 0.3]]

JJ = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

AA = [[0, 0, 0] ,[0, 0, 0], [0.99925, 0.625147, 0.551273]]
gama = [[0.0124425, 0.0806628, 0.00999575],[0.0550028, 0.0758354, 0.07346340],[0.0972069, 0.0723954, 0.07405450]]
# gama = 1
t = 2.24
E1 = (
    1 / 2 * sum([sum(i) for i in zip(*JJ)])
    + 1 / 2 * sum([sum(i) for i in zip(*gama)])
    + 1 / 2 * sum([sum(i) for i in zip(*AA)])
    + 1 / 2 * sum([sum(i) for i in zip(*extmagfield_n)])
    + 1 / 2 * extmagfield_m
)
tau = E1 * t / 2
print(E1)
kappa = 3 / 2 * tau
print(tau)

2.313364725
2.5909684920000005


In [657]:
def spin_op(N):
    """calculates spin oprators

    Args:
        N (int): number of electrons or atoms

    Returns:
        matrix: matrices of spin oprators
    """
    Sx = []
    Sy = []
    Sz = []

    for n in range(N):
        x_list = []
        y_list = []
        z_list = []

        for m in range(N):
            x_list.append(si)
            y_list.append(si)
            z_list.append(si)

        x_list[n] = sx
        y_list[n] = sy
        z_list[n] = sz

        Sx.append(tensor(x_list))
        Sy.append(tensor(y_list))
        Sz.append(tensor(z_list))

    return [Sx, Sy, Sz]

In [658]:
Sx, Sy, Sz = spin_op(m+n)
# HS & HB are bare hamiltonians of the central system and bath
# V is the system-bath interaction
HS = HB = V = 0
for i in range(m):
    HS += extmagfield_m * Sx[i] + extmagfield_m * Sy[i] + extmagfield_m * Sz[i]
    for j in range(m):
        HS += JJ[0][j] * Sx[i] * Sx[j] + JJ[1][j] * Sy[i] * Sy[j] + JJ[2][j] * Sz[i] * Sz[j]

for i in range(m,m+n):
    HB += extmagfield_n[0][i] * Sx[i] + extmagfield_n[1][i] * Sy[i] + extmagfield_n[2][i] * Sz[i]
    for j in range(m,m+n):
        HB += gama[0][j] * Sx[i] * Sx[j] + gama[1][j] * Sy[i] * Sy[j] + gama[2][j] * Sz[i] * Sz[j]
for i in range(m,m+n):
    for j in range(m):
        V += AA[0][j] * Sx[i] * Sx[j] + AA[1][j] * Sy[i] * Sy[j] + AA[2][j] * Sz[i] * Sz[j]
HH = HS + HB + V


In [659]:
HH





Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 2.5281436+0.j  0.       +0.j  1.       -1.j  0.       +0.j]
 [ 0.       +0.j -0.0703564+0.j  0.       +0.j  1.       -1.j]
 [ 1.       +1.j  0.       +0.j -1.4703564+0.j  0.       +0.j]
 [ 0.       +0.j  1.       +1.j  0.       +0.j -0.0718564+0.j]]

In [660]:
def TG(k, G):
    if k == 1:
        return 1
    elif k == 2:
        return G
    else:
        return 2 * G * TG(k - 1, G) - TG(k - 2, G)

In [661]:
G = 2 * HH / E1
UU = 0
for k in range(1, int(kappa) + 1):
    a = 1
    if k == 1:
        a = 2
    UU += a * ((1j) ** k) * jv(k, tau) * TG(k, G)



In [662]:
UU

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-1.00082487-1.75009559j  0.        +0.j         -0.76525437+0.02649249j
   0.        +0.j        ]
 [ 0.        +0.j          0.02785223+0.48010114j  0.        +0.j
  -0.34621249+0.44553436j]
 [-0.02649249-0.76525437j  0.        +0.j          0.58207503-0.2731259j
   0.        +0.j        ]
 [ 0.        +0.j         -0.44553436-0.34621249j  0.        +0.j
   0.02844604+0.48002665j]]

In [663]:
zero = basis(2) 
c = create(2)
one=c*zero
psi=tensor(one,zero)
psi0=psi*psi.dag()
print("trace: {}".format(psi0.tr())) 
psi0

trace: 1.0


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

In [665]:
# psif = UU * psi0
# result = psif * psif.dag()
# print("trace: {:.1f}".format(result.tr()))
# result
psif = UU * psi0 * UU.dag()
print("trace: {:.2f}".format(psif.tr()))
psif

trace: 1.00


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.5863161 +0.j          0.        +0.j         -0.45267124-0.19359017j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [-0.45267124+0.19359017j  0.        +0.j          0.41340909+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]]